# Cuaderno 22: Implementando algoritmos de plano cortante
## Problema del ordenamiento lineal 
## Linear Ordering Problem (LOP)

$\newcommand{\card}[1]{\left| #1 \right|}$
$\newcommand{\tabulatedset}[1]{\left\{ #1 \right\}}$
$\newcommand{\ZZ}{\mathbb{Z}}$
$\newcommand{\RR}{\mathbb{R}}$

Dados: 
* un grafo dirigido **completo** $D=(V,A)$, sin lazos; y,
* un vector $w \in \ZZ^{A}$ de pesos asociados a los arcos de $D$.

El *problema del ordenamiento lineal (Linear Ordering Problem, LOP)* consiste en encontrar un conjunto de arcos con las siguientes propiedades:
* entre cada par de nodos debe seleccionarse exactamente uno de los dos arcos (antiparalelos) posibles;
* los arcos seleccionados no deben formar circuitos; y,
* la suma de los pesos de los arcos seleccionados debe ser máxima.

Notar que la segunda condición equivale a requerir que los arcos seleccionados no formen circuitos de longitud 3, es decir, triángulos orientados.

Empleando variables binarias $x_{ij}$ para indicar selección de arcos, LOP puede formularse como el siguiente programa lineal entero:

\begin{align*}
\max &\sum_{(i,j) \in A} w_{ij} x_{ij}\\ 
& \mbox{s.r.}\\
& x_{ij} + x_{ji} = 1, \quad \forall i \neq j \in V,\\
& x_{ij} + x_{jk} + x_{ki} \leq 2, \forall i, j, k \in V \mbox{ distintos entre sí},\\
& x_{ji} + x_{ik} + x_{kj} \leq 2, \forall i, j, k \in V \mbox{ distintos entre sí},\\
& x_{ij} \in \{0, 1\}, \quad \forall (i,j) \in A.
\end{align*}

La función objetivo mide el peso total de los arcos seleccionados.

La primera familia de restricciones indica que debe seleccionarse exactamente uno de los dos arcos que conectan a cada par de nodos.

La segunda familia de restricciones prohibe que se seleccionen arcos que forman circuitos de longitud igual a tres. Como se selecciona un arco entre cada par de nodos, puede demostrarse que si los arcos seleccionados forman circuitos, entonces forman circuitos de longitud 3. Por lo tanto, esta familia de restricciones prohibe la formación de cualquier circuito.

Vamos a implementar este modelo usando la interfaz Python de Gurobi. Colocaremos las restricciones de arcos antiparalelos de la manera usual en el modelo, pero agregaremos las restricciones de eliminación de triángulos de manera dinámica durante el proceso de solución.


Definimos primero los datos. Usaremos la función `randint` del módulo random para generar valores aleatorios en el rango {-10,..,10} para los pesos de los arcos. 

In [ ]:
from gurobipy import *
import random as rm
import math 

# Numero de nodos del grafo
n = 15

# Nodos del grafo: {1,..,n}
V = tuplelist(range(1,n+1))

# Arcos
A = [(i,j) for i in V for j in V if i!=j]

# los pesos son valores aleatorios en el rango {-10, 10}
w = tupledict({(i,j) : rm.randint(-10, 10) for (i,j) in A})

print('*** Matriz de pesos ***')
print(''.join(["{:3}   ".format(i) for i in V]))
print('-'*60)
for i in V:
    L =[]
    for j in V:
        if i!=j:
            L.append("{:3}   ".format(w[i,j]))
        else:
            L.append("---   ")
    print(''.join(L))
        


Definimos el objeto modelo, las variables y la función objetivo. Las asignaciones `m._x = x` y `m._V = V` serán utilizadas más adelante.

In [ ]:
# Crear el objeto modelo
m = Model('LOP')

# Variables de seleccion de arcos
x = m.addVars(A, name="x", vtype=GRB.BINARY)
m._x = x
m._V = V

# Crear la funcion objetivo
m.setObjective(x.prod(w,'*'), GRB.MAXIMIZE)


Añadimos las restricciones de selección de arcos antiparalelos, que seleccionan exactamente un arco entre cada par de nodos:

In [ ]:
# Restricciones de selección de arcos antiparalelos
m.addConstrs((x[i,j] + x[j,i]==1 for i,j in A if i < j), "antiparalelos")


### Restricciones lazy y funciones callback
Las restricciones de eliminación de triángulos se implementarán como restricciones tipo *lazy*. Estas restricciones se agregan dinámicamente al modelo conforme son requeridas. Para ello, es necesario definir una función tipo **callback**.

Una función tipo callback es una función diseñada para ser llamada por el solver Gurobi durante el proceso de solución del modelo. Esta función puede tener cualquier nombre, pero debe recibir obligatoriamente dos parámetros: `model` y `where`. La función debe en primer lugar examinar el valor de `where` para determinar desde qué parte del proceso de solución ha sido llamada y tomar la acción que sea correspondiente. Esto se hace comparando este parámetro contra algunos *códigos callback*. En nuestro ejemplo, el código `GRB.Callback.MIPSOL` indica que se acaba de encontrar una nueva solución entera que satisface las demás restricciones del problema. En el manual de referencia puede consultarse la lista completa de [códigos callback](https://www.gurobi.com/documentation/9.5/refman/cb_codes.html).

El parámetro `model` proporciona acceso a las variables y métodos del objeto modelo. Esto es útil para acceder a ciertas funciones como `cbGetSolution`, que permite recuperar la solución actual; o `cbLazy` que permite agregar dinámicamente una restricción lazy al modelo. También se usa para tener acceso a variables y cualquier otra información del modelo. Para ello, en el programa principal, la información a compartir debe incluirse como una variable del objeto modelo. En en lenguaje Python, se acostumbra empezar los nombres de estas variables con el símbolo `_`. En nuestro ejemplo, empleamos `_x`para acceder de las variables de selección de arcos y `_V`para acceder a la lista con los vértices.

En el ejemplo, la función callback se activa únicamente cuando se ha encontrado una nueva solución entera que satisface todas las desigualdades de arcos antiparalelos. Llamando a `cbGetSolution(model._x)` se recuperan los valores de las variables $x_{ij}$ dentro de la solución actual y se almacenan en `vx`. Luego se itera sobre todos los conjuntos de tres nodos y se verifica que estos valores satisfagan las dos desigualdades triangulares posibles. Cada vez que se encuentra un triángulo donde la suma de los valores de las variables supera 2, se agrega la desigualdad correspondiente llamando a la función `cbLazy`.

In [ ]:
# Definir función callback
def mycallback(model, where):
    # Esta función se activará cuando se encuentre una nueva solución entera
    if where == GRB.Callback.MIPSOL:
        # Recuperar los valores de la solución actual
        print("Hola! Solución nueva encontrada")
        vx = model.cbGetSolution(model._x)
        # Determinar si existe desigualdad triangular que sea violada
        l = 0
        for i in model._V:
            for j in model._V:
                if i>=j: 
                    continue
                for k in model._V:
                    if j>=k: 
                        continue
                    
                    # Iteramos sobre todos los nodos i < j < k, y verificamos las dos restricciones posibles
                    if vx[i, j] + vx[j, k] + vx[k, i] > 2:
                        model.cbLazy(model._x[i, j] + model._x[j, k] 
                                                    + model._x[k, i] <=2)
                        l+= 1
                    if vx[i, k] + vx[k, j] + vx[j, i] > 2:
                        model.cbLazy(model._x[i, k] + model._x[k, j] 
                                           + model._x[j, i] <=2)
                        l+= 1
        print('Se agregaron {} desigualdades'.format(l))
        

Para poder utilizar restricciones tipo *lazy* en un modelo, debe fijarse el parámetro `LazyConstraints` al valor de 1.

In [ ]:
# Configurar Gurobi para usar restricciones lazy
m.Params.LazyConstraints = 1


Fijamos un tiempo límite de 180 segundos y una tolerancia para la brecha de optimalidad del 15%:

In [ ]:
# Terminar al alcanzar un Gap del 20%
m.Params.MIPGap = 0.2

# Terminar luego de 180 segundos
m.Params.TimeLimit = 180


Escribimos el modelo a un archivo de texto. Notar que el modelo inicial contiene solamente las restricciones de selección de arcos antiparalelos.

In [ ]:
# Escribir el modelo a un archivo
m.write('LOP-lazy.lp')


Finalmente, resolvemos el modelo y mostramos la solución. Al llamar a `optimize` es necesario pasar el nombre de la función *callback* como un parámetro.

In [ ]:
# Calcular la solucion optima
m.optimize(mycallback)

# Escribir la solucion
if m.SolCount > 0:
    # Recuperar los valores de las variables
    vx = m.getAttr('x', x)
    print('\nOrdenamiento óptimo:')
    for i,j in A:
        if vx[i,j] >= 0.9:
            print('{} -> {}'.format(i, j))


## Código completo

Se reproduce a continuación el código completo del modelo anterior.

In [ ]:
# Implementación de modelos lineales enteros
# Problema de ordenamiento lineal
# Formulación usando restricciones lazy 
# para las desigualdades de eliminación de triángulos

# Luis M. Torres (EPN 2022)

from gurobipy import *
import random as rm

# Definir función callback
def mycallback(model, where):
    # Esta función se activara cuando se encuentre una nueva solución entera
    if where == GRB.Callback.MIPSOL:
        # print("Hola! Solución nueva encontrada")
        # Recuperar los valores de la solución actual
        vx = model.cbGetSolution(model._x)
        # Determinar si existe desigualdad triangular que sea violada
        for i in model._V:
            for j in model._V:
                if i>=j: continue
                for k in model._V:
                    if j>=k: continue
                    # Iteramos sobre todos los nodos i < j < k, y verificamos las dos restricciones posibles
                    if vx[i, j] + vx[j, k] + vx[k, i] > 2:
                        model.cbLazy(model._x[i, j] + model._x[j, k] + model._x[k, i] <=2)
                    if vx[i, k] + vx[k, j] + vx[j, i] > 2:
                        model.cbLazy(model._x[i, k] + model._x[k, j] + model._x[j, i] <=2)


# Número de nodos del grafo
n = 50

# Nodos del grafo: {1,..,n}
V = tuplelist(range(1,n+1))

# Arcos
A = [(i,j) for i in V for j in V if i!=j]

# Los pesos son valores aleatorios en el rango {-10, 10}
w = tupledict({(i,j) : rm.randint(-10, 10) for (i,j) in A})

# Escribir la matriz con los datos ingresados
print('*** Matriz de pesos ***')
print(''.join(["{:3}   ".format(i) for i in V]))
print('-'*60)
for i in V:
    L =[]
    for j in V:
        if i!=j:
            L.append("{:3}   ".format(w[i,j]))
        else:
            L.append("---   ")
    print(''.join(L))


try:
    # Crear el objeto modelo
    m = Model('LOP')

    # Variables de selección de arcos
    x = m.addVars(A, name="x", vtype=GRB.BINARY)

    # Crear la función objetivo
    m.setObjective(x.prod(w,'*'), GRB.MAXIMIZE)

    # Crear variables en el objeto modelo para el acceso 
    # a las variables de selección de arcos y al conjunto de nodos
    m._x = x
    m._V = V

    # Restricciones de selección de arcos antiparalelos
    m.addConstrs((x[i,j] + x[j,i]==1 for i,j in A if i < j), "antiparalelos")

    # Escribir el modelo a un archivo
    m.write('LOP-lazy.lp')

    # Configurar Gurobi para usar restricciones lazy
    m.Params.LazyConstraints = 1

    # Terminar al alcanzar un Gap del 20%
    m.Params.MIPGap = 0.2

    # Fijar tiempo límite en 180 segundos
    m.Params.TimeLimit = 180

    # Iniciar algoritmo de solución
    m.optimize(mycallback)

    # Escribir la solución
    if m.SolCount > 0:
        # Recuperar los valores de las variables
        vx = m.getAttr('x', x)
        print('\nOrdenamiento óptimo:')
        for i,j in A:
            if vx[i,j] > 0:
                print('{} -> {}'.format(i, j))
                
except GurobiError as e:
    print('Se produjo un error de Gurobi: código: ' + str(e.errno) + ": " + str(e))

except AttributeError:
    print('Se produjo un error de atributo')